In [ ]:
import pandas as pd
import vectorbt as vbt
import pandas_ta as ta
import datetime
import time
import MetaTrader5 as mt5


if not mt5.initialize():
    print("initialize() failed, error code =", mt5.last_error())
    quit()
symbol = "XAUUSD"


while True:
    print("Getting data in progress...")
    # Get the latest data
    bars = mt5.copy_rates_from(symbol, mt5.TIMEFRAME_M1, datetime.datetime.now(), 365)
    df = pd.DataFrame(bars)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df = df.set_index('time')
    current_price = df['close'].iloc[-1]

    # Calculate indicators
    print("Calculating indicators in progress...")
    ma14 = vbt.MA.run(df['close'], 14)
    ma50 = vbt.MA.run(df['close'], 50)
    ma365 = vbt.MA.run(df['close'], 365)
    rsi = vbt.RSI.run(df['close'], 14)
    
    # Check the conditions for the last bar
    print("Checking conditions in progress...\n")
    if (ma14.ma.iloc[-1] > ma50.ma.iloc[-1] > ma365.ma.iloc[-1] and rsi.rsi.iloc[-1] < 40):
        print("Buy condition met:")
        print(f"RSI: {rsi.rsi.iloc[-1]:.2f} (below 40)")
        print(f"14 SMA: {ma14.ma.iloc[-1]:.5f} (above 50 SMA and 50 SMA > 365 SMA)")
        print(f"Current price: {current_price:.5f}")
        print("-" * 30)
    
    elif (ma14.ma.iloc[-1] < ma50.ma.iloc[-1] < ma365.ma.iloc[-1] and rsi.rsi.iloc[-1] > 60):
        print("Sell condition met:")
        print(f"RSI: {rsi.rsi.iloc[-1]:.2f} (above 60)")
        print(f"14 SMA: {ma14.ma.iloc[-1]:.5f} (below 50 SMA and 50 SMA < 365 SMA)")
        print(f"Current price: {current_price:.5f}")
        print("-" * 30)

    time.sleep(60)  # wait for 60 seconds
